## Setup

Import TensorFlow and other necessary libraries:

In [1]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

## Load data using a Keras utility

Next, load these images off disk using the helpful tf.keras.utils.image_dataset_from_directory utility.
This will take you from a directory of images on disk to a tf.data.Dataset in just a couple lines of code.

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = (180, 180)

In [4]:
# Creation of the train, validation and test dataset
# The split for train, validation and test is 80 - 16 - 4

data_dir = "../data/raw"

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE)

class_names = train_ds.class_names
print(class_names)

Found 60992 files belonging to 51 classes.
Using 48794 files for training.
Found 60992 files belonging to 51 classes.
Using 12198 files for validation.
['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Background_without_leaves', 'Black-grass', 'Blueberry___healthy', 'Charlock', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Fat Hen', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Loose Silky-bent', 'Maize', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill',

As the original dataset doesn't contain a test set, you will create one.
To do so, determine how many batches of data are available in the validation set using `tf.data.experimental.cardinality`, then move 20% of them to a test set.

In [5]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5) # 4% of entire dataset
val_ds = val_ds.skip(val_batches // 5) # 16% of entire dataset

print('Number of training batches: %d' % tf.data.experimental.cardinality(train_ds))
print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_ds))

Number of training batches: 1525
Number of validation batches: 306


## Configure the dataset for performance

Make sure to use buffered prefetching, so you can yield data from disk without having I/O become blocking.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Transfer Learning approach

Create the base model from the pre-trained model MobileNetV2

In [ ]:
# This model expects pixel values in [-1, 1], but at this point, the pixel values in your images are in [0, 255]
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

base_model = MobileNetV2(input_shape=(180, 180, 3),
                         include_top=False,
                         weights='imagenet')

image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

# Freeze the base model
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
# Add a classification head
global_average_layer = layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = layers.Dense(len(class_names), activation='softmax')

In [ ]:
# Build the model
inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
layer = preprocess_input(inputs)
layer = base_model(layer, training=False)
layer = global_average_layer(layer)
layer = tf.keras.layers.Dropout(0.2)(layer)
outputs = prediction_layer(layer)

model = tf.keras.Model(inputs, outputs)

## Compile the model

Compile the model before training it. Since there are many classes and a softmax oputput, use the `categorical_crossentropy`.

In [ ]:
base_learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Train the model

After training for 10 epochs, you should see ~96% accuracy on the validation set.

In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(val_ds)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 10, verbose = 1, restore_best_weights = True)
lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.2, patience = 3, verbose = 1, cooldown = 5)

history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds,
                    callbacks = [early_stopping, lr_plateau])

## Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNetV2 base model as a fixed feature extractor.

In [ ]:
import matplotlib.pyplot as plt


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Fine tuning

In the feature extraction experiment, you were only training a few layers on top of an MobileNetV2 base model. The weights of the pre-trained network were not updated during training.

In [ ]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

## Compile the model

As you are training a much larger model and want to readapt the pretrained weights, it is important to use a lower learning rate at this stage. Otherwise, your model could overfit very quickly.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Continue training the model

If you trained to convergence earlier, this step will improve your accuracy by a few percentage points.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         validation_data=val_ds,
                         callbacks = [early_stopping, lr_plateau])

### Saving the model

In [ ]:
# Saving the model
model.save("../TL_180px_32b_20e_model.keras")

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Evaluation and prediction

Finally you can verify the performance of the model on new data using test set.

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = test_ds.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

In [ ]:
# Loading the model
model = tf.keras.models.load_model("../models/TL_180px_32b_20e_model.keras")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Creating the predictions for the test_ds
predictions = np.array([])
labels =  np.array([])

for x, y in test_ds:
  predictions = np.concatenate([predictions, np.argmax(model.predict(x, verbose = False), axis = -1)]).astype(int)
  labels = np.concatenate([labels, y.numpy()]).astype(int)

# Creating a scatter plot on F1 score for the test-ds in relation to support, also add a line for the weighted average
precision, recall, fscore, support = precision_recall_fscore_support(labels, predictions)

f1score = classification_report(labels, predictions, output_dict=True)["weighted avg"]["f1-score"]

# Create the scatterplot
plt.scatter(support, fscore, marker='o', color='b')

plt.plot((0, support.max()), (f1score, f1score),
         label = 'average F1 Score',
         color = 'green')

plt.xlabel("Support")
plt.ylabel("F1 Score")
plt.title("F1 Score vs Support")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Print the classification report, for this we "translate" the labels
c_true = np.array(class_names)[labels]
c_predict = np.array(class_names)[predictions]

print("Transfer Learning")
print(classification_report(c_true, c_predict))